In [1]:
import sys
import os
import re
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
import pandas as pd
import plotly.express as px
import numpy as np
import geopandas as gpd
from pyproj import Transformer
from scripts.utils import split_column_at
from scripts.accent_cleaner import AccentCleaner
#from deep_translator import GoogleTranslator

### Connectivity

In [2]:
df_connectivity = pd.read_csv("../data/raw/conectivity_municipality.csv", encoding="ISO-8859-1", sep=";", decimal=",")

df_connectivity.drop(["Community", "Province"], axis=1, inplace = True)
df_connectivity.rename(columns={"housing": "number_of_houses", "municipality_code": "cmun"}, inplace=True)


df_connectivity.query("municipality == 'Murcia'")


,cmun,municipality,population,number_of_houses,VDSL_30Mbps,fixed_wireless,FTTH,HFC,reception_30Mbps,reception_100Mbps,reception_1Gbps,4G,5G,"5G_3,5GHz"
4595,30030,Murcia,462.979,213.291,"7,97%","99,97%","97,38%","64,56%","97,86%","94,72%","86,58%","99,91%","97,60%","83,13%"


In [3]:
df_connectivity.columns


Index(['cmun', 'municipality', 'population', 'number_of_houses', 'VDSL_30Mbps',
       'fixed_wireless', 'FTTH', 'HFC', 'reception_30Mbps',
       'reception_100Mbps', 'reception_1Gbps', '4G', '5G', '5G_3,5GHz'],
      dtype='object')

In [4]:
percentage_columns = ['VDSL_30Mbps', 'fixed_wireless', 'FTTH', 'HFC', 'reception_30Mbps','reception_100Mbps', 'reception_1Gbps', '4G', '5G', '5G_3,5GHz']

df_connectivity[percentage_columns] = df_connectivity[percentage_columns].applymap(
    lambda x: round(float(x.rstrip('%').replace(',', '.')) / 100, 2)
)

df_connectivity


C:\Users\Marina\AppData\Local\Temp\ipykernel_20316\1345967164.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_connectivity[percentage_columns] = df_connectivity[percentage_columns].applymap(


,cmun,municipality,population,number_of_houses,VDSL_30Mbps,fixed_wireless,FTTH,HFC,reception_30Mbps,reception_100Mbps,reception_1Gbps,4G,5G,"5G_3,5GHz"
0,1001,Alegría-Dulantzi,2.960,1.275,0.29,1.00,0.99,0.0,1.00,0.81,0.15,1.00,0.01,0.0
1,1002,Amurrio,10.281,4.937,0.10,1.00,0.92,0.0,0.93,0.87,0.89,1.00,0.96,0.0
2,1003,Aramaio,1.431,738,0.21,0.94,0.85,0.0,0.89,0.78,0.84,1.00,0.60,0.0
3,1004,Artziniega,1.810,1.099,0.11,1.00,0.87,0.0,0.98,0.95,0.97,1.00,0.84,0.0
4,1006,Armiñón,235,180,0.00,0.53,0.87,0.0,0.99,0.84,0.90,0.96,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8126,14902,La Guijarrosa,1.341,643,0.00,0.95,0.78,0.0,0.89,0.89,0.86,1.00,0.05,0.0
8127,18077,Fornes,519,357,0.00,1.00,0.95,0.0,0.99,0.95,0.95,1.00,0.31,0.0
8128,18916,Torrenueva Costa,3.038,5.583,0.00,1.00,0.98,0.0,1.00,0.98,0.97,1.00,1.00,1.0
8129,21902,La Zarza-Perrunal,1.230,1.086,0.00,1.00,1.00,0.0,1.00,1.00,0.88,1.00,1.00,0.0


In [5]:
if 'population' in df_connectivity.columns:
    df_connectivity.drop(columns=["population"], inplace=True)

In [6]:
df_connectivity['number_of_houses'] = df_connectivity['number_of_houses'].str.replace('.', '').astype('float64')

In [7]:
df_connectivity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8131 entries, 0 to 8130
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   cmun               8131 non-null   int64  
 1   municipality       8131 non-null   object 
 2   number_of_houses   8131 non-null   float64
 3   VDSL_30Mbps        8131 non-null   float64
 4   fixed_wireless     8131 non-null   float64
 5   FTTH               8131 non-null   float64
 6   HFC                8131 non-null   float64
 7   reception_30Mbps   8131 non-null   float64
 8   reception_100Mbps  8131 non-null   float64
 9   reception_1Gbps    8131 non-null   float64
 10  4G                 8131 non-null   float64
 11  5G                 8131 non-null   float64
 12  5G_3,5GHz          8131 non-null   float64
dtypes: float64(11), int64(1), object(1)
memory usage: 825.9+ KB


In [8]:
df_connectivity.to_csv("../data/processed/filtered_files/filtered_connectivity.csv", index=False)